In [13]:
import os
from pathlib import Path
import random

import puppets_data
import ConnectionGraphX
import GraphX

import numpy as np
import pandas as pd
import scipy
import networkx as nx

import ConnectionNetworkX as cnx

from pyLDLE2 import datasets
from pyLDLE2 import buml_
from pyLDLE2 import util_

from proxi.algorithms.knng import get_knn_graph
from proxi.utils.misc import save_graph, save_weighted_graph
from proxi.utils.process import *
from proxi.utils.distance import abs_correlation

In [14]:
p = "/home/sawyerjack/IdeaProjects/data-git-ignore/PuppetsData"
save_dir_root = '/home/sawyerjack/IdeaProjects/connection-graph-wasserstein/data'

## Just for time and sanity, using n = 100 while I test things.
N_IMAGES = 1000

In [15]:
Y, labelsMat, _ = puppets_data.puppets_data(p)

X.shape =  (8100, 100)


In [16]:
X = np.array(Y.copy())
totalImages = X.shape[0]
sampleIndices = random.sample(range(totalImages), N_IMAGES)
X = [ X[i] for i in sampleIndices ]

X = np.array(X)
X.shape

(1000, 100)

In [17]:
buml_obj = buml_.BUML(local_opts={'algo':'LPCA', 'k': 28},
                      vis_opts={'c': labelsMat[:,0], 'save_dir': save_dir_root},
                      verbose=True, debug=True, exit_at='local_views')

buml_obj.fit(X=X)


local_opts['k_nn0'] = 700 is created.
Options provided:
local_opts:
{
    "Atilde_method": "LDLE_1",
    "N": 100,
    "U_method": "k_nn",
    "algo": "LPCA",
    "alpha": 1,
    "debug": true,
    "delta": 0.9,
    "gl_type": "unnorm",
    "k": 28,
    "k_nn": 49,
    "k_nn0": 700,
    "k_tune": 7,
    "lambda1_decay": 0.75,
    "lambda1_init": 8,
    "lambda1_min": 0.001,
    "max_iter": 300,
    "max_sparsity": 0.9,
    "metric": "euclidean",
    "n_proc": 6,
    "p": 0.99,
    "power": 5,
    "pp_n_thresh": 32,
    "radius": 0.5,
    "reg": 0.0,
    "scale_by": "gamma",
    "tau": 50,
    "to_postprocess": true,
    "verbose": true
}
intermed_opts:
{
    "algo": "best",
    "debug": true,
    "eta_max": 25,
    "eta_min": 5,
    "len_S_thresh": 256,
    "local_algo": "LPCA",
    "metric": "euclidean",
    "n_proc": 6,
    "n_times": 4,
    "verbose": true
}
global_opts:
{
    "add_dim": false,
    "align_transform": "rigid",
    "align_w_parent_only": true,
    "alpha": 0.3,
    "b

In [18]:
INTRINSIC_DIMENSION = 2

directedPuppetGraph = nx.DiGraph(buml_obj.LocalViews.U)
a = nx.adjacency_matrix(directedPuppetGraph.to_undirected())
puppetGraph = cnx.ConnectionNetworkX(a, INTRINSIC_DIMENSION)

TypeError: The numpy boolean negative, the `-` operator, is not supported, use the `~` operator or the logical_not function instead.

In [ ]:
for i in range(N_IMAGES):
    n_i = nx.neighbors(puppetGraph, i)
    for j in [j for j in n_i if j > i]:
        n_ij = buml_obj.LocalViews.U[i,:].multiply(buml_obj.LocalViews.U[j,:]).nonzero()[1]
        X_Uij_i = buml_obj.LocalViews.local_param_post.eval_({'view_index': i, 'data_mask': n_ij})
        X_Uij_j = buml_obj.LocalViews.local_param_post.eval_({'view_index': j, 'data_mask': n_ij})
        Tij, _ = util_.custom_procrustes(X_Uij_i, X_Uij_j)

        puppetGraph.updateEdgeSignature((i,j), Tij)
